In [4]:
%%cmd
pip install --upgrade pip
pip install -r requirements.txt 

Microsoft Windows [Version 10.0.19045.3324]
(c) Microsoft Corporation. All rights reserved.

(venv310) d:\Documents\work\docqa>pip install --upgrade pip
  Using cached pip-23.2.1-py3-none-any.whl (2.1 MB)


ERROR: To modify pip, please run the following command:
D:\Documents\work\ask_api\ask-api\venv310\Scripts\python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



(venv310) d:\Documents\work\docqa>pip install -r requirements.txt 
     ---------------------------------------- 2.1/2.1 MB 2.5 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.3.0
    Uninstalling pydantic-2.3.0:
      Successfully uninstalled pydantic-2.3.0



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



(venv310) d:\Documents\work\docqa>

In [22]:
%jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


check virtual environment

In [1]:
import sys
sys.prefix

'd:\\Documents\\work\\ask_api\\ask-api\\venv310'

In [3]:
import os
import glob
import pprint
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
import openai
import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions
from tqdm.notebook import tqdm




openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
import langchain
langchain.__version__

'0.0.274'

In [1]:
import pydantic 
pydantic.__version__

'1.10.8'

In [33]:
chromadb_client = chromadb.Client(Settings(
    chroma_db_impl='duckdb+parquet', #chroma_db_impl="duckdb",
    persist_directory="./chroma_db/"
))
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
collection_ncc2022 = chromadb_client.get_or_create_collection(name="ncc2022", embedding_function=sentence_transformer_ef) 

# openai_ef = embedding_functions.OpenAIEmbeddingFunction(
#                 api_key=os.environ['OPENAI_API_KEY'],
#                 model_name="text-embedding-ada-002"
#             )

# collection_ncc_openai = client.create_collection(name="ncc2022openai", embedding_function=openai_ef) 

Using embedded DuckDB with persistence: data will be stored in: ./chroma_db/


In [36]:
chromadb_client.list_collections()

No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


[Collection(name=ncc2022)]

In [22]:
#chromadb_client.delete_collection(name="ncc2022")

In [35]:
file_folder = "D:/Documents/data/national construction code/2022/*.pdf"
for f in glob.glob(file_folder):
    print(f'processing {f}')
    pdf_loader = PyPDFLoader(f)
    docs = pdf_loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
    docs = text_splitter.split_documents(docs)
    collection_ncc2022.add(
        documents=[d.page_content for d in docs],
        metadatas=[d.metadata for d in docs],
        ids=[d.metadata['source'] + str(d.metadata['page']) for d in docs]
    )
chromadb_client.persist()

processing D:/Documents/data/national construction code/2022\abcb-housing-provisions-2022-20230501.pdf
processing D:/Documents/data/national construction code/2022\ncc2022-combined-vol2-housing-provisions-20230501.pdf
processing D:/Documents/data/national construction code/2022\ncc2022-consolidated-performance-requirements-20230501.pdf
processing D:/Documents/data/national construction code/2022\ncc2022-volume-one-20230501.pdf
processing D:/Documents/data/national construction code/2022\ncc2022-volume-three-20230501.pdf
processing D:/Documents/data/national construction code/2022\ncc2022-volume-two-20230501.pdf


 impossible to decode XFormObject /Fm0


True

In [43]:
chromadb_client_2 = chromadb.Client(Settings(
    chroma_db_impl="duckdb+parquet",
    persist_directory="D:/Documents/work/docqa/chroma_db"
))
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
collection_ncc2022_2 = chromadb_client_2.get_or_create_collection(name="ncc2022", embedding_function=sentence_transformer_ef) 


Using embedded DuckDB with persistence: data will be stored in: D:/Documents/work/docqa/chroma_db


In [44]:
collection_ncc2022_2.query(query_texts=["What's the maximum travel distance between two fire exits in a shopping center?"],
                     include=['documents', 'distances', 'metadatas'],
                     n_results=3)

{'ids': [['D:/Documents/data/national construction code/2022\\ncc2022-volume-one-20230501.pdf185',
   'D:/Documents/data/national construction code/2022\\ncc2022-volume-one-20230501.pdf193',
   'D:/Documents/data/national construction code/2022\\ncc2022-combined-vol2-housing-provisions-20230501.pdf513']],
 'embeddings': None,
 'documents': [['Access and egress\nNCC 2022 Volume One - Building Code of Australia Page 186must be not more than 60 m. \n(6) Assembly buildings — In a Class 9b building other than a school  or early childhood centre , the distance to one of the  \nexits may be 60 m if—  \nthe path of travel from the room concerned to that exit is through another area which is a corridor, hallway, lobby,  (a)\nramp or other circulation space; and  \nthe room is smoke-separated from the circulation space by construction having an FRL of not less than 60/60/60  (b)\nwith every doorway in that construction protected by a tight fitting, self-closing , solid-core door not less than 35

In [42]:
chromadb_client_3 = chromadb.Client(Settings(
    chroma_db_impl="duckdb+parquet",
    persist_directory="D:/Documents/work/docqa/chroma_db"
))
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
collection_ncc2022_3 = chromadb_client_3.get_or_create_collection(name="ncc2022", embedding_function=sentence_transformer_ef) 
# file_folder = "D:/Documents/data/national construction code/2022/*.pdf"
# for f in glob.glob(file_folder):
#     print(f'processing {f}')
#     pdf_loader = PyPDFLoader(f)
#     docs = pdf_loader.load()
#     text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
#     docs = text_splitter.split_documents(docs)
#     collection_ncc2022.add(
#         documents=[d.page_content for d in docs],
#         metadatas=[d.metadata for d in docs],
#         ids=[d.metadata['source'] + str(d.metadata['page']) for d in docs]
#     )
# chromadb_client.persist()
print(chromadb_client_3.list_collections())
text = collection_ncc2022_3.query(query_texts=["What's the maximum travel distance between two fire exits in a shopping center?"],
                     include=['documents', 'distances', 'metadatas'],
                     n_results=3)
text

Using embedded DuckDB with persistence: data will be stored in: D:/Documents/work/docqa/chroma_db
No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


[Collection(name=ncc2022)]


{'ids': [['D:/Documents/data/national construction code/2022\\ncc2022-volume-one-20230501.pdf185',
   'D:/Documents/data/national construction code/2022\\ncc2022-volume-one-20230501.pdf193',
   'D:/Documents/data/national construction code/2022\\ncc2022-combined-vol2-housing-provisions-20230501.pdf513']],
 'embeddings': None,
 'documents': [['Access and egress\nNCC 2022 Volume One - Building Code of Australia Page 186must be not more than 60 m. \n(6) Assembly buildings — In a Class 9b building other than a school  or early childhood centre , the distance to one of the  \nexits may be 60 m if—  \nthe path of travel from the room concerned to that exit is through another area which is a corridor, hallway, lobby,  (a)\nramp or other circulation space; and  \nthe room is smoke-separated from the circulation space by construction having an FRL of not less than 60/60/60  (b)\nwith every doorway in that construction protected by a tight fitting, self-closing , solid-core door not less than 35

In [37]:
text = collection_ncc2022.query(query_texts=["What's the maximum travel distance between two fire exits in a shopping center?"],
                     include=['documents', 'distances', 'metadatas'],
                     n_results=3)
print(text)

{'ids': [['D:/Documents/data/national construction code/2022\\ncc2022-volume-one-20230501.pdf185', 'D:/Documents/data/national construction code/2022\\ncc2022-volume-one-20230501.pdf193', 'D:/Documents/data/national construction code/2022\\ncc2022-combined-vol2-housing-provisions-20230501.pdf513']], 'embeddings': None, 'documents': [['Access and egress\nNCC 2022 Volume One - Building Code of Australia Page 186must be not more than 60 m. \n(6) Assembly buildings — In a Class 9b building other than a school  or early childhood centre , the distance to one of the  \nexits may be 60 m if—  \nthe path of travel from the room concerned to that exit is through another area which is a corridor, hallway, lobby,  (a)\nramp or other circulation space; and  \nthe room is smoke-separated from the circulation space by construction having an FRL of not less than 60/60/60  (b)\nwith every doorway in that construction protected by a tight fitting, self-closing , solid-core door not less than 35  \nmm t

In [13]:
def build_prompt_with_context(question, context):
    system_prompt = {
        "role": "system",
        "content": """You are a helpful assistant who knows a lot about architecture standards in Australia. 
        I will ask you questions about architecture standards. I will provide a number of relevant paragraphs extracted from 
        the architecture standards for your reference."""
    }
    user_prompt = {
        "role": "user",
        "content": f"""
        Please help me on the question delimited by ``` regarding archituecture standards in Australia. The texts delimited by === are 
        a number of paragraphs extracted from National Construction Code. You can use the texts as reference. Please provide your reply in format of json with 3 fields: "answer",
        "confidence score" between 0 to 1, and "source" which are the original clauses supporting your answer. 
        
        Question:```
        {question}
        ```
        Texts:===
        {os.linesep.join(context)}
        ===
        """
    }
    return [system_prompt,
            user_prompt]





In [8]:
question = "What class of building does a shopping center belong to? What distance should fire staires of shopping center be?"
texts = collection_ncc2022.query(query_texts=[question],
                     include=['documents', 'distances', 'metadatas'],
                     n_results=3
                     )
messages = build_prompt_with_context(question, texts['documents'][0])
response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',
            messages=messages,
            max_tokens=1000,
        )
response

<OpenAIObject chat.completion id=chatcmpl-7DPEUY9XE2HKBMpyIo47h1aYYB9n9 at 0x17d1034cae0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Based on the National Construction Code, a shopping center would fall under the Class 6 category in Australia, as it is a commercial building that serves more than one tenant. Regarding fire stairs, if a Class 6 building has a single storey, a fire-isolated stairway serving the Class 6 should have an automatic smoke detection and alarm system complying with Specification 20. If a Class 6 building has more than one storey, air pressurisation system must be fitted for the fire-isolated exits. The answer confidence is 0.9 as the source clearly mentions that shopping center belongs to Class 6, but it does not explicitly mention the required distance for fire stairs. \n\nAnswer: \n{\n    \"answer\": \"A shopping center belongs to Class 6 in the building classification in Australia. Fire 

In [44]:
texts

{'ids': [['D:/Documents/data/national construction code\\NCC_2019_BCA_Guide.pdf217',
   'D:/Documents/data/national construction code\\NCC_2019_BCA_Guide.pdf93',
   'D:/Documents/data/national construction code\\ncc2022-volume-one-20230501.pdf245']],
 'embeddings': None,
 'documents': [['Services and equipment \nNCC 2019 Guide to BCA Volume One Page 218 Figure E2.2b(1) Plan of shopping centre for Example 1 \nThe specific smoke hazard management provisions for Class 6 buildings containing shops and an enclosed common \nwalkway or mall are similar. Where such a building has a fire compartment with a floor area of more than 2 000 m 2, an \nautomatic smoke exhaust system or an automatic smoke-and-heat vent system to control smoke during a fire is required. \nHowever, where a single storey shop does not open onto the mall and its main public entrance opens to a road or open \nspace, it does not need to have any smoke exhaust or smoke venting, if its floor area does not exceed 2 000 m 2. Thi

In [45]:
question = "What distance should fire staires of shopping center be?"
texts = collection_ncc2022.query(query_texts=[question],
                     include=['documents', 'distances', 'metadatas'],
                     n_results=3
                     )
messages = build_prompt_with_context(question, texts['documents'][0])
response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',
            messages=messages,
            max_tokens=1000,
        )
response

<OpenAIObject chat.completion id=chatcmpl-7DBmgsxthsbyrBtt5OxfXPUhrz4fb at 0x1909f935120> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The required distance for fire stairs within a shopping center in Australia is not explicitly stated in the given text. However, the NCC 2019 Guide to BCA Volume One provides information on the width of stairways and ramps where it states that the maximum width of the stairway should be limited to 2 meters, which allows for a reach of 1 meter to either side of the barrier. Wider stairways are allowed but are deemed to add no more than 2 meters of aggregate width. The guide also specifies safe gradients for ramps used as required exits. In case of a shopping center, the required distance for fire stairs would be determined based on the use and size of the building, as well as other factors such as building materials, fire safety systems, among others. Therefore, it is recommended tha

In [33]:
question = "What class of building does a shopping center belong to?"
texts = collection_ncc2022.query(query_texts=[question],
                     include=['documents', 'distances', 'metadatas'],
                     n_results=3
                     )
messages = build_prompt_with_context(question, texts['documents'][0])
response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',
            messages=messages,
            max_tokens=1000,
        )
response

<OpenAIObject chat.completion id=chatcmpl-7DAg1VCQvHiaA9TdhOTMmWmnjJolO at 0x1909b0c9260> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Answer: \nA shopping center belongs to Class 6 of the building classification according to the National Construction Code in Australia. Class 6 includes buildings that are for the sale of goods by retail or the supply of services directly to the public, including eating rooms, cafes, restaurants, and service stations. \n\nConfidence Score: 0.95\n\nSource: \n- A3.2 General Provisions of the National Construction Code (NCC) 2015 Building Code of Australia - Volume One. Specifically, Class 6 is defined as \"a shop or other building for the sale of goods by retail or the supply of services direct to the public, including\u2014(a) an eating room, caf\u00e9, restaurant, milk or soft-drink bar; or (b) a dining room, bar area that is not an assembly building, shop or kiosk part of a hotel o

In [46]:
question = "What length of fire stairways of shopping centers should be?"
texts = collection_ncc2022.query(query_texts=[question],
                     include=['documents', 'distances', 'metadatas'],
                     n_results=3
                     )
messages = build_prompt_with_context(question, texts['documents'][0])
response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',
            messages=messages,
            max_tokens=1000,
        )
response

<OpenAIObject chat.completion id=chatcmpl-7DBnevqFAJMrCky1JuIXWcLgQfxev at 0x1909b0de750> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Based on the NCC 2019 Guide to BCA Volume One, the required width for stairways is a maximum of 2 meters, effectively permitting a reach of 1 meter to either side. Wider stairways are permitted, with more than 2 meters between the handrails, but are deemed to add no more than 2 meters to the aggregate required width. There are, however, no specific length requirements for fire stairways of shopping centers in Australia mentioned in this text. Therefore, I cannot provide a confident answer to your question.",
        "role": "assistant"
      }
    }
  ],
  "created": 1683377978,
  "id": "chatcmpl-7DBnevqFAJMrCky1JuIXWcLgQfxev",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 111,
    "prompt_tokens": 1075,
    "total_tokens": 118

In [20]:
question = "What's the maximum travel distance between two fire exits in a shopping center?"
texts = collection_ncc2022.query(query_texts=[question],
                     include=['documents', 'distances', 'metadatas'],
                     n_results=3
                     )
messages = build_prompt_with_context(question, texts['documents'][0])
response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',
            messages=messages,
            max_tokens=1000,
        )
response

<OpenAIObject chat.completion id=chatcmpl-7EGMHxVMEqZm2OQSdLmlF53P7kgEl at 0x158394acf90> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Based on the text, the maximum travel distance allowed between two fire exits in a shopping center is 60 meters, as stated in SA D2D6. The maximum distance may only exceed 60 meters if specific requirements are met, such as the room being smoke-separated from the circulation space by a construction with an FRL of not less than 60/60/60, and the maximum distance of travel not exceeding 40 meters within the room and 20 meters from the doorway. The confidence score for this answer is high since the text clearly states the distance between alternative fire exits for Class 2 and 3 buildings and in health-care buildings (45 meters apart) and for all other cases (60 meters apart). \n\n{\n    \"answer\": \"The maximum travel distance allowed between two fire exits in a shopping center is 60

In [48]:
texts

{'ids': [['D:/Documents/data/national construction code\\NCC_2019_BCA_Guide.pdf129',
   'D:/Documents/data/national construction code\\NCC_2019_BCA_Guide.pdf128',
   'D:/Documents/data/national construction code\\NCC_2019_BCA_Guide.pdf136']],
 'embeddings': None,
 'documents': [['Access and egress \nNCC 2019 Guide to BCA Volume One Page 130 blocked. \nUniform distribution of exits— D1.5(a) \nD1.5(a) requires that where multiple exits exist, they must be distributed as uniformly as practicable to improve the level \nof safety when evacuating. \nMinimum and maximum distances are set \nD1.5 specifies the minimum and maximum permitted distances between alternative exits (for example, two exits may be \nlocated next to one another, so long as they are not each other’s alternative in an emergency). Where scissor stairs are \nused, the shafts will be adjacent and separated by fire-resisting construction. However, the access doors to the alternative \nscissor stairs must comply with the minimu

[Document(page_content='2022Housing Provisions \nStandard', metadata={'source': 'D:\\Documents\\data\\national construction code\\2022\\abcb-housing-provisions-2022-20230501.pdf', 'page': 0}),
 Document(page_content='Front matter\nABCB Housing Provisions Standard 2022 Page 2  \n \n© Commonwealth of Australia and the States and Territories of Australia 2022, published by the Australian Building Codes  \nBoard \n \n \nThe material in this publication is licensed under a Creative Commons Attribution-NoDerivatives - 4.0 International licence,  \nwith the exception of: \nany third party material  •\nany trade marks, and •\nany images or photographs.  •\nYou may not make derivatives of this publication, but may only use a verbatim copy. More information on this CC BY ND  \nlicense is set out at the Creative Commons Website. Enquiries about this publication can be sent to:  \nAustralian Building Codes Board  \nGPO Box 2013, Canberra ACT 2601  \nPhone: 1300 134 631  \nEmail: ncc@abcb.gov.au  \

In [11]:
# splitted_documents[0]

Document(page_content='2022Housing Provisions \nStandard', metadata={'source': 'D:\\Documents\\data\\national construction code\\2022\\abcb-housing-provisions-2022-20230501.pdf', 'page': 0})

In [12]:
# raw_documents[0]

Document(page_content='2022Housing Provisions \nStandard', metadata={'source': 'D:\\Documents\\data\\national construction code\\2022\\abcb-housing-provisions-2022-20230501.pdf', 'page': 0})

### Create langchian vectorstore backed by chroma db from documents

In [1]:
from langchain.embeddings import SentenceTransformerEmbeddings

from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

raw_documents = PyPDFDirectoryLoader("D:/Documents/data/national construction code/2022/").load()

print(f"raw_documents {len(raw_documents)}")
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=500,
                                                                     chunk_overlap=20,
                                                                    )
splitted_documents = text_splitter.split_documents(raw_documents)

print(f"splitted docuemnts {len(splitted_documents)}")

chroma_db = Chroma.from_documents(
    collection_name="building_code",
    persist_directory="./chroma_db_2",
    documents=splitted_documents, 
    embedding=SentenceTransformerEmbeddings()
    )
print("loaded splitted documents to chromadb")
chroma_db.persist()
print("persisted chroma db")
print(chroma_db.count())

 impossible to decode XFormObject /Fm0


raw_documents 3239
splitted docuemnts 5316


KeyboardInterrupt: 

### Load existing langchain chroma db

In [29]:
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
chroma_db = Chroma(
    collection_name="building_code",
    persist_directory="./chroma_db_2",
    embedding_function=SentenceTransformerEmbeddings(),
    )

In [30]:
question = "What's the maximum travel distance between two fire exits in a shopping center?"


texts = chroma.max_marginal_relevance_search(query=question, n_results=3)

In [31]:
texts

[]

In [10]:
from typing import List
from langchain import LLMChain
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.chat_models import ChatOpenAI

# Output parser will split the LLM result into a list of queries
class LineList(BaseModel):
    # "lines" is the key (attribute name) of the parsed output
    lines: List[str] = Field(description="Lines of text")


class LineListOutputParser(PydanticOutputParser):
    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        lines = text.strip().split("\n")
        return LineList(lines=lines)


output_parser = LineListOutputParser()

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five 
    different versions of the given user question to retrieve relevant documents from a vector 
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search. 
    Provide these alternative questions seperated by newlines.
    Original question: {question}""",
)
llm = ChatOpenAI(temperature=0)

# Chain
llm_chain = LLMChain(llm=llm, prompt=QUERY_PROMPT, output_parser=output_parser)

# Other inputs
question = "What's the maximum travel distance between two fire exits in a shopping center?"

In [14]:
llm_chain.run(question)

LineList(lines=['1. What is the maximum allowable distance between two fire exits in a shopping center?', '2. Is there a specific limit on the travel distance between fire exits in shopping centers?', '3. How far apart can fire exits be located in a shopping center?', '4. Are there any regulations regarding the maximum travel distance between fire exits in shopping centers?', '5. What is the maximum distance that can separate two fire exits in a shopping center?'])

In [15]:
from langchain.retrievers.multi_query import MultiQueryRetriever
# Run
retriever = MultiQueryRetriever(
    retriever=chroma.as_retriever(), llm_chain=llm_chain, parser_key="lines"
)  # "lines" is the key (attribute name) of the parsed output

# Results
unique_docs = retriever.get_relevant_documents(
    query=question
)

In [17]:
unique_docs

[]

In [19]:
chroma_retriever = chroma.as_retriever()
chroma_retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], metadata=None, vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x000002957107C9A0>, search_type='similarity', search_kwargs={})

In [27]:
question

"What's the maximum travel distance between two fire exits in a shopping center?"

In [26]:
chroma_retriever.vectorstore._collection.query(query_texts=[question],
                     include=['documents', 'distances', 'metadatas'],
                     n_results=3
                     )

{'ids': [[]],
 'embeddings': None,
 'documents': [[]],
 'metadatas': [[]],
 'distances': [[]]}

In [32]:
chroma_retriever.vectorstore._collection.peek()

{'ids': [], 'embeddings': [], 'documents': [], 'metadatas': []}